### Convert body key points to its original image size body key points

#### Input an image and its key points txt file, return the original image size body key points as a new txt file

In [28]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
import glob
import json

scaledX = 600;
scaledY = 400;

In [4]:
# define a function input the scaled image key points return the original image key points as a np ndarray
# scaleX, scaleY: size of the scaled image
# oriX, oriY: size of the original image
def getOriginalKeyPoints(scaleX, scaleY, scalePoints, oriX, oriY):
    numRow = scalePoints.shape[0]; # 13 for 1 person
    numCol = scalePoints.shape[1]; # 4 columns: person index, key point index, x, y
    temp = np.empty([numRow, numCol]);
    temp[:,0:2] = scalePoints[:,0:2];
    for row in range(0, numRow):
        for i in range(2,4):
            temp_i = scalePoints[row][i]
            temp_i = temp_i[temp_i.find(':')+1:]
            if temp_i == 'N/A':
                temp[row][i] = np.NaN;
            else:
                if i == 2:
#                     update x position of the original image
                    temp[row][i] = int(float(temp_i) * (oriX / scaleX)); 
                else:
#                     update y position of the original image
                    temp[row][i] = int(float(temp_i) * (oriY / scaleY)); 
    return temp;

In [23]:
# define a function input a directory, which contains original images and the corresponding scaled key points
# return and save all the original key points in a new folder named 'originalTxt'
# directory: the directory, where original images and the scaled key points stored. Sample:'Image&txt/'
# dType: save the original key points as either json or txt format, by typing 'json' and 'txt' respectively
def getAllOriKeyPoints(directory, dType):
    listImage = sorted(glob.glob(directory + '*.jpg'));
    listTxt = sorted(glob.glob(directory + '*.txt'));
    for i in range(0, len(listImage)):
        img = cv2.imread(listImage[i]);
        Ysize = img.shape[0];
        Xsize = img.shape[1];
        file = np.loadtxt(listTxt[i], dtype='str', delimiter=',', unpack=True, usecols=range(4));
        scalePoints = np.transpose(file);
        oriKeyPoints = getOriginalKeyPoints(scaledX, scaledY, scalePoints, Xsize, Ysize);
        if dType == 'txt':
            np.savetxt('originalTxt' + listTxt[i][listTxt[i].find('/'):], oriKeyPoints, delimiter=',', fmt='%.2f');
        else:
            j1 = json.loads(json.dumps(oriKeyPoints.tolist()));
            with open('originalTxt' + listTxt[i][listTxt[i].find('/'):listTxt[i].find('.')] + '.json', 'w') as outfile:
                json.dump(j1, outfile)
        

In [24]:

# define a function input the original image and its original key points
# print those key points on the given image
def printKeyPoints(image, keyPoints): 
    new = cv2.circle(image, (int(keyPoints[0][2]),int(keyPoints[0][3])), 5, (0,0,255), 3)
    for i in range(1,13):
        if not np.isnan(keyPoints[i][2]):
            new = cv2.circle(new, (int(keyPoints[i][2]),int(keyPoints[i][3])), 5, (0,0,255),3)
    cv2.imshow('image',new)


### Tests

In [25]:
#Test for getAllOriKeyPoints function
# save all key points as json files in the originalTxt folder
getAllOriKeyPoints('Image&txt/', 'json')

In [8]:
# Test for getOriginalKeyPoints function
file = np.loadtxt('Image&txt/XOTGz-1vizY_00032.txt', dtype='str', delimiter=',', unpack=True, usecols=range(4))
file1 = np.transpose(file);
list1 = getOriginalKeyPoints(600, 400, file1, Xsize, Ysize)
print(list1)
# np.savetxt('originalTxt/test.txt', list1, delimiter=',', fmt='%.2f')

[[0.000e+00 0.000e+00 1.082e+03 4.730e+02]
 [0.000e+00 1.000e+00 1.020e+03 4.980e+02]
 [0.000e+00 2.000e+00 9.120e+02 5.200e+02]
 [0.000e+00 3.000e+00 8.260e+02 5.340e+02]
 [0.000e+00 4.000e+00 8.160e+02 4.800e+02]
 [0.000e+00 5.000e+00 6.830e+02 3.450e+02]
 [0.000e+00 6.000e+00 7.280e+02 4.950e+02]
 [0.000e+00 7.000e+00       nan       nan]
 [0.000e+00 8.000e+00       nan       nan]
 [0.000e+00 9.000e+00       nan       nan]
 [0.000e+00 1.000e+01       nan       nan]
 [0.000e+00 1.100e+01       nan       nan]
 [0.000e+00 1.200e+01       nan       nan]]


In [26]:
#Test for printKeyPoints function
image1 = cv2.imread(sorted(glob.glob('Image&txt/' + '*.jpg'))[0]);
Ysize = image1.shape[0];
Xsize = image1.shape[1];
scaledKeyPoints1 = sorted(glob.glob('Image&txt/' + '*.txt'))[0];
scaledKeyPoints1 = np.loadtxt(scaledKeyPoints1, dtype='str', delimiter=',', unpack=True, usecols=range(4));
scaledKeyPoints1 = np.transpose(scaledKeyPoints1);

oriKeyPoints1 = getOriginalKeyPoints(600, 400, scaledKeyPoints1, Xsize, Ysize);

printKeyPoints(image1, oriKeyPoints1);

#cv2.destroyAllWindows()